# 8 Reviewing video annotations in Label Studio

[Label Studio](https://labelstud.io/) is a tool for creating training data for machine learning. It is browser based and can be used to annotate video, images and other data. 

We will use it to review the annotations we made in the previous steps. 


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ultralytics
import fiftyone as fo
import logging
import sys

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
# Add project root to path and import utils
project_root = os.path.join("..")
sys.path.append(project_root)

from src.config import PATH_CONFIG
from src.utils.io_utils import getProcessedVideos, saveProcessedVideos, getFaceData, getSpeechData, getKeyPoints, getVideoProperty
from src.utils.notebook_utils import display_config_info, ensure_dir_exists
from src.utils.keypoint_utils import normalize_keypoints
from src.processors.keypoint_processor import process_keypoints_for_modeling
from src.processors.face_processor import normalize_facial_keypoints, match_faces_to_poses

# Get paths from config
videos_in = PATH_CONFIG['videos_in']
data_out = PATH_CONFIG['data_out']

# Ensure output directory exists
if ensure_dir_exists(data_out):
    print(f"Created output directory: {data_out}")

# Display configuration information
display_config_info(videos_in, data_out, "Processing Configuration")

# Use the configured filename from PATH_CONFIG
processedvideos = getProcessedVideos(data_out)
processedvideos.head()

2025-04-10 10:05:27,410 - WARNING - c:\Users\caspar\Anaconda3\envs\babyjokes\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

2025-04-10 10:05:32,658 - WARNING - From c:\Users\caspar\Anaconda3\envs\babyjokes\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

2025-04-10 10:05:32,658 - WARNING - From c:\Users\caspar\Anaconda3\envs\babyjokes\Lib\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.




## Processing Configuration
    
| Configuration | Value | Status |
|---------------|-------|--------|
| Input Videos | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\LookitLaughter.test` | ✅ exists |
| Output Data | `c:\Users\caspar\OneDrive\LegoGPI\babyjokes\data\1_interim` | ✅ exists |
| Video Count | 54 videos | |

You can change these paths by modifying the `PATH_CONFIG` in `src/config.py` 
or by overriding them in this notebook.


Found existing processed_videos.csv with 54 rows.


,VideoID,ChildID,JokeType,JokeNum,JokeRep,JokeTake,HowFunny,LaughYesNo,Frames,FPS,...,Speech.file,Diary.file,Diary.when,LastError,annotatedVideo,annotated.when,FrameCount,Keypoints.normed,Face_Processing_Complete,Faces.normed
0,2UWdXP.joke1.rep2.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,2,1,Slightly funny,No,NaN,14.232999,...,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,2025-04-03 20:16:15,NaN,NaN,NaN,216,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...,True,..\data\1_interim\2UWdXP.joke1.rep2.take1.Peek...
1,2UWdXP.joke1.rep3.take1.Peekaboo_h265.mp4,2UWdXP,Peekaboo,1,3,1,Slightly funny,No,NaN,14.263979,...,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,2025-04-03 20:16:17,NaN,NaN,NaN,150,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...,True,..\data\1_interim\2UWdXP.joke1.rep3.take1.Peek...
2,2UWdXP.joke2.rep1.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,1,1,Funny,No,NaN,12.275790,...,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,2025-04-03 20:16:18,NaN,NaN,NaN,89,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep1.take1.NomN...
3,2UWdXP.joke2.rep2.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,2,1,Slightly funny,No,NaN,13.920731,...,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,2025-04-03 20:16:19,NaN,NaN,NaN,95,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep2.take1.NomN...
4,2UWdXP.joke2.rep3.take1.NomNomNom_h265.mp4,2UWdXP,NomNomNom,2,3,1,Slightly funny,No,NaN,14.010793,...,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,2025-04-03 20:16:20,NaN,NaN,NaN,132,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...,True,..\data\1_interim\2UWdXP.joke2.rep3.take1.NomN...


## 8.1 Launch Label Studio

Let's build the command to launch Label Studio inside a docker container.
Getting the folder paths correct is critical for video access. There are two main approaches:

1. **Mount your video folder directly** - Label Studio can access videos through mounted volumes
2. **Upload videos through API** - Upload files directly to Label Studio storage

### Mounting Video Folders

For Label Studio to find your videos, the path in the task data must match the path inside the container.
Here's how to set up the correct volumes:

- Map your local video folder to `/label-studio/data/videos` inside the container
- Map a folder for Label Studio's own storage to `/label-studio/data/storage`

❗ **Important**: The path to videos in tasks must be `/data/local-files/?d=/label-studio/data/videos/your_file.mp4`

In [ ]:
# Configuration for Label Studio Docker setup
import os
import subprocess

# Replace with your video directory - keep this ABSOLUTE, not relative
VIDEO_DIR = os.path.abspath(os.path.join(project_root, "LookitLaughter.test"))
STORAGE_DIR = os.path.abspath(os.path.join(project_root, "labelstudio_data"))

# Ensure the storage directory exists
os.makedirs(STORAGE_DIR, exist_ok=True)

# For Windows, convert backslashes to forward slashes for Docker
VIDEO_DIR = VIDEO_DIR.replace('\\', '/')
STORAGE_DIR = STORAGE_DIR.replace('\\', '/')

print(f"Video directory: {VIDEO_DIR}")
print(f"Storage directory: {STORAGE_DIR}")

# Build Docker run command
command = [
    'docker', 'run', '-d',
    '-p', "8080:8080",
    '-e', "LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED=true",
    '-e', "LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=/label-studio/data",
    # Map video directory for direct access
    '-v', f"{VIDEO_DIR}:/label-studio/data/videos",
    # Map storage directory for Label Studio's data
    '-v', f"{STORAGE_DIR}:/label-studio/data/storage",
    '--name', "labelstudio",
    'heartexlabs/label-studio:latest'
]

# Join the command parts correctly
command_str = ' '.join(command)

print(f"\nRunning command: {command_str}")

try:
    # Use shell=True for Windows and pass the command as a string
    result = subprocess.run(command_str, check=True, shell=True, capture_output=True, text=True)
    print("\nLabel Studio container started. Access at http://localhost:8080")
    print("Container ID:", result.stdout.strip())
except subprocess.CalledProcessError as e:
    print(f"Error executing docker command: {e}")
    print(f"Error output: {e.stderr}")


Running command: docker run -d -p 8080:8080 -e LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED=true -e LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT=/label-studio -v C:/Users/Caspar/OneDrive/ls/data:/label-studio/data -v C:/Users/Caspar/OneDrive/ls/files:/label-studio/files --name labelstudio heartexlabs/label-studio:latest
Label Studio container started. Container ID:
e7fd01879ecd2271332bc73a4c65951508a3e33598d1fd7b8996ceae4582bda6

Access Label Studio at http://localhost:8080
Label Studio container started. Container ID:
e7fd01879ecd2271332bc73a4c65951508a3e33598d1fd7b8996ceae4582bda6

Access Label Studio at http://localhost:8080


In [5]:
# To check the logs from the Label Studio container
log_command = "docker logs labelstudio"

try:
    logs = subprocess.run(log_command, shell=True, capture_output=True, text=True, check=True)
    print(logs.stdout)
except subprocess.CalledProcessError as e:
    print(f"Error getting logs: {e.stderr}")

April 10, 2025 - 09:06:51
Django version 5.1.6, using settings 'core.settings.label_studio'
Starting development server at http://0.0.0.0:8080/
Quit the server with CONTROL-C.

redirect to home page



## Container Management

If you need to stop, restart, or remove the container, you can use these commands:

In [ ]:
# To stop the Label Studio container
# !docker stop labelstudio

# To restart the Label Studio container
# !docker start labelstudio

# To remove the container (will lose any data not in mounted volumes)
# !docker rm -f labelstudio

Next we connect to the container and start Label Studio. 
To find your api key:

1. Docker
You can go to the command line inside Docker Desktop and run the following command:

```bash
docker exec -it label-studio bash
```

2. Label Studio

You can also find the API key in the Label Studio UI. Go to your profile settings [Top Right of screen] and go to account and settings. Look for the API key section. It should be displayed there.

In [6]:
# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = 'c7fa731be79261a59379e210785b0e32f375cda4'

# Import the SDK and the client module
from label_studio_sdk.client import LabelStudio

# Connect to the Label Studio API and check the connection
ls = LabelStudio(base_url=LABEL_STUDIO_URL, api_key=API_KEY)

print("Connected to Label Studio API")

# List all projects to verify the connection
projects = ls.projects.list()
if not projects:
    print("No projects found in Label Studio.") 
print("Projects in Label Studio:")
for project in projects.items:
    print(f"Project ID: {project.id}, Name: {project.title}")

2025-04-10 10:08:39,836 - INFO - HTTP Request: GET http://localhost:8080/api/projects/?page=1 "HTTP/1.1 200 OK"


Connected to Label Studio API
Projects in Label Studio:
Project ID: 2, Name: babyjokes
Project ID: 1, Name: webm


## 8.2 Create and Configure a Project for Baby Jokes

Now we'll check if we have a project called 'babyjokes', and if not, we'll create it and import our videos.

In [7]:
# Function to check if the project already exists and return its ID if found
def get_project_by_name(project_name):
    projects = ls.projects.list()
    for project in projects.items:
        if project.title == project_name:
            return project.id
    return None

# Check if babyjokes project exists
project_name = "babyjokes"
project_id = get_project_by_name(project_name)

if project_id:
    print(f"Project '{project_name}' already exists with ID: {project_id}")
    # Get the existing project
    project = ls.projects.get(project_id)
else:
    print(f"Project '{project_name}' doesn't exist. Creating now...")
    
    # Define a labeling configuration for video annotation with our categories
    label_config = """<View>
      <Header value="Baby Jokes Video Annotation"/>
      <Video name="video" value="$video" muted="false"/>
      
      <!-- Joke type classification -->
      <Choices name="joke_type" toName="video" choice="single">
        <Choice value="Peekaboo"/>
        <Choice value="TearingPaper"/>
        <Choice value="NomNomNom"/>
        <Choice value="ThatsNotAHat"/>
        <Choice value="ThatsNotACat"/>
      </Choices>
      
      <!-- Funniness rating -->
      <Choices name="funniness" toName="video" choice="single">
        <Choice value="Not Funny"/>
        <Choice value="Slightly Funny"/>
        <Choice value="Funny"/>
        <Choice value="Extremely Funny"/>
      </Choices>
      
      <!-- Laughter detection -->
      <Choices name="laughed" toName="video" choice="single">
        <Choice value="Yes"/>
        <Choice value="No"/>
      </Choices>
      
      <!-- Temporal regions for events within the video -->
      <Labels name="events" toName="video">
        <Label value="Speech" background="#FFA39E"/>
        <Label value="Laughter" background="#D4380D"/>
        <Label value="Joke" background="#FFC069"/>
        <Label value="Reaction" background="#7CB305"/>
      </Labels>
    </View>"""
    
    # Create the project
    project = ls.projects.create(
        title=project_name,
        description="Video annotation for baby jokes analysis",
        label_config=label_config
    )
    
    print(f"Created new project with ID: {project.id}")
    project_id = project.id

print(f"Using project: {project_name} (ID: {project_id})")

2025-04-10 10:16:53,960 - INFO - HTTP Request: GET http://localhost:8080/api/projects/?page=1 "HTTP/1.1 200 OK"
2025-04-10 10:16:53,997 - INFO - HTTP Request: GET http://localhost:8080/api/projects/2/ "HTTP/1.1 200 OK"
2025-04-10 10:16:53,997 - INFO - HTTP Request: GET http://localhost:8080/api/projects/2/ "HTTP/1.1 200 OK"


Project 'babyjokes' already exists with ID: 2
Using project: babyjokes (ID: 2)


## 8.2 Video Import Options

There are two ways to make videos accessible to Label Studio:

1. **Using Local Files** - Reference files in mounted volumes (setup above)
2. **Direct Upload** - Upload files directly to Label Studio

Let's implement both methods to give users flexibility.

In [ ]:
import os
import time
from pathlib import Path

def upload_video_to_labelstudio(file_path, project_id, ls_client):
    """Upload a video file directly to Label Studio instead of referencing local files"""
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            return None

        print(f"Uploading {os.path.basename(file_path)} to Label Studio...")
        file_name = os.path.basename(file_path)
        
        # Open the file and upload it
        with open(file_path, 'rb') as f:
            # Upload the file to the specified project
            upload_response = ls_client.upload_file(f, file_name, project_id)
            
            # If successful, the response will include the file upload ID
            if upload_response:
                print(f"File uploaded successfully. ID: {upload_response.id}")
                
                # The uploaded file path will look like: /data/upload/{project_id}/{file_id}-{filename}
                # Create a task that references this file
                task = ls_client.tasks.create(
                    project=project_id,
                    data={
                        "video": upload_response.url
                    }
                )
                print(f"Task created with ID: {task.id}")
                return upload_response, task
            else:
                print("Upload failed.")
                return None
    except Exception as e:
        print(f"Error uploading file: {e}")
        print(traceback.format_exc())
        return None

def upload_all_videos(video_folder, project_id, ls_client):
    """Upload all videos from a folder to Label Studio"""
    results = []
    video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mov'))][0:5]  # Limit to 5 for testing
    
    print(f"Found {len(video_files)} video files in {video_folder}")
    
    for video_file in video_files:
        file_path = os.path.join(video_folder, video_file)
        print(f"\nUploading {video_file}...")
        result = upload_video_to_labelstudio(file_path, project_id, ls_client)
        results.append((video_file, result))
        time.sleep(1)  # Slight pause between uploads to avoid overloading the server
        
    print(f"\nUploaded {len(results)} videos to project {project_id}")
    return results

In [ ]:
# Function to prepare video data for Label Studio import
def prepare_video_tasks(video_df, use_local_files=True):
    tasks = []
    
    for i, row in video_df.iterrows():
        try:
            video_path = row['video_path']
            # Check if the path is valid
            if not os.path.exists(video_path):
                print(f"WARNING: Video file not found: {video_path}")
                continue
                
            # Get the video filename
            video_filename = os.path.basename(video_path)
            
            # Depending on how we're serving files, create the right URL
            if use_local_files:
                # Path for accessing files via local file serving
                # This needs to match the Docker volume mounting
                # For our Docker setup, videos are mounted at /label-studio/data/videos
                video_url = f"/data/local-files/?d=/label-studio/data/videos/{video_filename}"
            else:
                # If using upload instead, this will be replaced later
                # Use a placeholder that indicates an upload is needed
                video_url = "UPLOAD_NEEDED"
            
            # Create task data with the video URL and any existing metadata
            task_data = {
                "video": video_url,
                "video_filename": video_filename,
                "video_original_path": video_path,  # Store original path for upload fallback
                "metadata": {}
            }
            
            # Add any other metadata from the DataFrame
            for column in video_df.columns:
                if column not in ['video_path']:
                    task_data["metadata"][column] = str(row[column])
            
            tasks.append({
                "data": task_data
            })
            
        except Exception as e:
            print(f"Error processing video at index {i}: {e}")
    
    return tasks

In [9]:
import traceback

# Get tasks using the proper SDK method
try:
    # Use the SDK's tasks.list method to get tasks for the project
    tasks_response = ls.tasks.list(project=project_id)
    
    if tasks_response and hasattr(tasks_response, 'items'):
        existing_tasks = tasks_response.items
        print(f"Project already has {len(existing_tasks)} tasks.")
    else:
        print("No tasks found for this project.")
        existing_tasks = []
    
    # Only import videos if the project is empty
    if len(existing_tasks) == 0:
        # Extract video information from processedvideos
        if 'video_path' not in processedvideos.columns:
            print("Adding video_path column to processedvideos DataFrame")
            # Construct video paths based on videos_in directory and video names
            processedvideos['video_path'] = processedvideos['VideoID'].apply(
                lambda x: os.path.join(videos_in, x) if x.endswith('.mp4') else os.path.join(videos_in, f"{x}.mp4")
            )
        
        # Prepare tasks for import
        video_tasks = prepare_video_tasks(processedvideos)
        print(f"Prepared {len(video_tasks)} videos for import.")
        
        # Import tasks into the project using the SDK
        if video_tasks:
            try:
                # Create tasks individually using the SDK
                imported_count = 0
                for task_data in video_tasks:
                    try:
                        # Create task using SDK method
                        response = ls.tasks.create(
                            project=project_id,
                            data=task_data['data']
                        )
                        imported_count += 1
                    except Exception as task_error:
                        print(f"Error importing video: {task_error}")
                
                print(f"Successfully imported {imported_count} of {len(video_tasks)} videos to the project.")
            except Exception as e:
                print(f"Error importing videos: {e}")
                print(traceback.format_exc())  # Print full traceback for debugging
        else:
            print("No valid videos found to import.")
    else:
        print("Project already has tasks. Skipping import to avoid duplicates.")
        print("If you want to reimport, please delete the existing tasks first.")
except Exception as e:
    print(f"Error accessing tasks: {e}")
    print(traceback.format_exc())  # Print full traceback for debugging

2025-04-10 10:17:04,687 - INFO - HTTP Request: GET http://localhost:8080/api/tasks/?page=1&project=2&fields=all "HTTP/1.1 200 OK"
2025-04-10 10:17:04,799 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:04,799 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:04,862 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:04,862 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"


Project already has 0 tasks.
Adding video_path column to processedvideos DataFrame
Prepared 54 videos for import.


2025-04-10 10:17:04,926 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:04,987 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:04,987 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,036 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,036 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,100 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,100 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,169 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,169 - INFO - HTTP Request: POST http://localhost:8080/api/tasks/ "HTTP/1.1 201 Created"
2025-04-10 10:17:05,525 - IN

Successfully imported 54 of 54 videos to the project.


## 8.3 Project Management

Here are some useful functions for managing your Label Studio project:

In [ ]:
def list_project_tasks(project_id, limit=10):
    """List tasks in a project using SDK methods"""
    try:
        # Use the SDK's tasks.list method
        tasks_response = ls.tasks.list(project=project_id)
        project_tasks = tasks_response.items
        
        if not project_tasks:
            print("No tasks found in this project")
            return []
            
        print(f"Total tasks in project {project_id}: {len(project_tasks)}")
        print("\nSample of tasks:")
        for i, task in enumerate(project_tasks[:limit]):
            filename = task.data.get('video_filename', 'Unnamed')
            print(f"Task {i+1}: ID={task.id}, {filename}")
        return project_tasks
    except Exception as e:
        print(f"Error listing tasks: {e}")
        print(traceback.format_exc())  # Print full traceback
        return []

def delete_task(task_id):
    """Delete a specific task"""
    try:
        # Use the SDK's tasks.delete method
        ls.tasks.delete(task_id)
        print(f"Task {task_id} deleted successfully")
        return True
    except Exception as e:
        print(f"Error deleting task {task_id}: {e}")
        print(traceback.format_exc())  # Print full traceback
        return False

def delete_project_tasks(project_id, confirm=False):
    """Delete all tasks in a project"""
    if not confirm:
        print("This will delete ALL tasks in the project.")
        print("Set confirm=True to proceed.")
        return
    
    try:
        # Get tasks for the project
        tasks_response = ls.tasks.list(project=project_id)
        project_tasks = tasks_response.items
        
        if not project_tasks:
            print("No tasks found in this project")
            return
        
        # Delete each task
        success_count = 0
        for task in project_tasks:
            try:
                ls.tasks.delete(task.id)
                success_count += 1
            except Exception as e:
                print(f"Failed to delete task {task.id}: {e}")
            
        print(f"Successfully deleted {success_count} of {len(project_tasks)} tasks from project ID {project_id}")
    except Exception as e:
        print(f"Error deleting tasks: {e}")
        print(traceback.format_exc())  # Print full traceback

# List all projects for reference
print("Available projects:")
for project in ls.projects.list().items:
    print(f"ID: {project.id}, Title: {project.title}")

# Usage example:
print("\nTo list tasks in the current project:")
print("list_project_tasks(project_id)")

print("\nTo delete all tasks in the project (use with caution):")
print("delete_project_tasks(project_id, confirm=True)")

Now let's see if we can add our metadata classifications. Recalling that each video demos one joke type `[Peekaboo,TearingPaper,NomNomNom,ThatsNotAHat,ThatsNotACat]` and has rating of how funny the baby found it `[Not Funny, Slightly Funny, Funny, Extremely Funny]` and whether they laughed `[Yes, No]`.


Let's add the frame by frame annotations directly onto the videos inside fiftyone

#### Add the speech as temporal annotations 

In [ ]:
def framerange_from_timestamps(timestamps, fps, max_frames):
    start = max(int(timestamps[0]*fps)+1 ,1)
    end =  min(int(timestamps[1]*fps)+1, max_frames )
    return start, end


In [ ]:
for sample in dataset:
    videoname = os.path.basename(sample.filepath)
    fps = sample.metadata["frame_rate"]
    max_frames = sample.metadata["total_frame_count"]
    print(fps)
    speechdata = utils.getSpeechData(processedvideos,videoname)
    if speechdata is None:
        print(f"Speech data not found for {videoname}")
        continue
    phrases = []
    for phrase in speechdata["segments"]:
        start, end = framerange_from_timestamps([phrase["start"],phrase["end"]], fps, max_frames)
        print (start, end)
        phrases.append(fo.TemporalDetection(label=phrase["text"],
                                        support=[start,end]))
        print(phrase["text"])
        
    sample["Speech"] = fo.TemporalDetections(detections=phrases)
    sample["Speech"] = phrases
    sample.save()

dataset.save()

In [ ]:
sample = dataset.first()


dets =[
        fo.TemporalDetection(label="meeting", support=[10, 20]),
        fo.TemporalDetection(label="party", support=[30, 60]),
    ]

sample["events"] = fo.TemporalDetections(
    detections= dets
)

print(sample)

In [ ]:
sample.save()

In [ ]:
for sample in dataset:
    videoname = os.path.basename(sample.filepath)
    speechdata = utils.getSpeechData(processedvideos,videoname)
    if speechdata is None:
        print(f"Speech data not found for {videoname}")
        continue
    
    subtitles = speechdata["segments"]
    # Create a list of text annotations
    text_annotations = [
        fo.Detection(
            text=sub["text"],
            start_time=sub["start"],
            end_time =sub["end"]
        )
        for sub in subtitles
    ]
    sample["subtitles"] = fo.Detections(detections=text_annotations)    
    sample.save()

14.299


In [ ]:
print(session.selected)

In [ ]:
#session.selected contains the indices of the dataset samples clicked on in the UI.
if len(session.selected) == 0:
    print("No samples selected. Click the checkbox in the top left of each video to select it.")
else:
    print(dataset[session.selected[0]])

## 8.4 Loading AI Annotations into Label Studio

Now let's create functions to load our automated annotations from various AI tools (YOLO, DeepFace, Whisper, Pyannote) into Label Studio for visualization.

In [ ]:
# Function to prepare speech annotations from Whisper results
def prepare_speech_annotations(video_id):
    """Convert speech transcription data from Whisper to Label Studio format"""
    try:
        # Get speech data from processed videos
        speech_data = getSpeechData(processedvideos, video_id)
        if not speech_data:
            print(f"No speech data found for {video_id}")
            return []
            
        # Get video duration to calculate relative time positions
        video_duration = getVideoProperty(processedvideos, video_id, "Duration")
        if not video_duration:
            # Try to get duration another way or use a default
            print(f"Warning: Could not get video duration for {video_id}")
            video_duration = 100.0  # Default to 100 seconds
            
        results = []
        # Process each speech segment
        for segment in speech_data.get('segments', []):
            start_time = segment.get('start', 0)
            end_time = segment.get('end', 0)
            text = segment.get('text', '')
            
            # Calculate relative start/end times (0-100%) for Label Studio
            start_percent = (start_time / video_duration) * 100 if video_duration else 0
            end_percent = (end_time / video_duration) * 100 if video_duration else 0
            
            # Create annotation in Label Studio format for speech transcription
            annotation = {
                "id": f"speech_{start_time}_{end_time}",
                "from_name": "events",  # Must match the tag name in our config
                "to_name": "video",    # Must match the tag name in our config
                "type": "labels",
                "value": {
                    "start": start_percent,
                    "end": end_percent,
                    "labels": ["Speech"],
                    "text": [text]
                }
            }
            results.append(annotation)
            
        print(f"Prepared {len(results)} speech annotations for {video_id}")
        return results
    except Exception as e:
        print(f"Error preparing speech annotations for {video_id}: {e}")
        return []

In [ ]:
# Function to prepare pose annotations from YOLO results
def prepare_pose_annotations(video_id):
    """Convert pose data from YOLO to Label Studio format"""
    try:
        # Get keypoint data from processed videos
        keypoint_data = getKeyPoints(processedvideos, video_id)
        if not keypoint_data or keypoint_data.empty:
            print(f"No keypoint data found for {video_id}")
            return []
            
        # Get video duration and FPS for time calculations
        fps = getVideoProperty(processedvideos, video_id, "FPS")
        if not fps:
            print(f"Warning: Could not get FPS for {video_id}")
            fps = 30  # Default value
            
        # Get unique frames where we have poses detected
        frame_data = keypoint_data.groupby(['frame', 'person']).count().reset_index()
        results = []
        
        # For Label Studio, we'll create temporal regions for sequences where people are present
        # This is a simplified representation - for detailed pose visualization may need more complex approach
        for person_type in keypoint_data['person'].unique():
            person_frames = keypoint_data[keypoint_data['person'] == person_type]['frame'].unique()
            
            if len(person_frames) > 0:
                # Calculate time ranges where the person is visible
                # Create a temporal annotation for each person showing when they're in frame
                min_frame = min(person_frames)
                max_frame = max(person_frames)
                
                start_time = min_frame / fps
                end_time = max_frame / fps
                
                annotation = {
                    "id": f"pose_{person_type}_{min_frame}_{max_frame}",
                    "from_name": "events",
                    "to_name": "video",
                    "type": "labels",
                    "value": {
                        "start": (start_time / end_time) * 100 if end_time else 0,  # As percentage
                        "end": 100.0,  # End percentage
                        "labels": [f"Person: {person_type}"],
                        "text": [f"Detected pose: {person_type}"]
                    }
                }
                results.append(annotation)
                
        print(f"Prepared {len(results)} pose annotations for {video_id}")
        return results
    except Exception as e:
        print(f"Error preparing pose annotations for {video_id}: {e}")
        return []

In [ ]:
# Function to prepare face emotion annotations from DeepFace results
def prepare_face_annotations(video_id):
    """Convert face emotion data from DeepFace to Label Studio format"""
    try:
        # Get face emotion data from processed videos
        face_data = getFaceData(processedvideos, video_id)
        if not face_data or face_data.empty:
            print(f"No face data found for {video_id}")
            return []
            
        # Get video duration and FPS for time calculations
        fps = getVideoProperty(processedvideos, video_id, "FPS")
        duration = getVideoProperty(processedvideos, video_id, "Duration")
        if not fps:
            print(f"Warning: Could not get FPS for {video_id}")
            fps = 30  # Default value
            
        results = []
        
        # Group emotions by person and emotion
        # For each continuous segment where a person shows the same emotion, create a temporal annotation
        for person_idx in face_data['index'].unique():
            person_data = face_data[face_data['index'] == person_idx]
            
            # Group by emotion
            for emotion, emotion_data in person_data.groupby('emotion'):
                if len(emotion_data) > 0:
                    frames = emotion_data['frame'].values
                    # Find continuous segments with the same emotion
                    segments = []
                    current_segment = [frames[0]]
                    
                    for i in range(1, len(frames)):
                        if frames[i] - frames[i-1] <= 5:  # Allow small gaps of 5 frames
                            current_segment.append(frames[i])
                        else:
                            segments.append(current_segment)
                            current_segment = [frames[i]]
                            
                    segments.append(current_segment)
                    
                    # Create a temporal annotation for each continuous segment
                    for segment in segments:
                        if len(segment) > 5:  # Only include significant segments (at least 5 frames)
                            start_frame = min(segment)
                            end_frame = max(segment)
                            
                            start_time = start_frame / fps
                            end_time = end_frame / fps
                            
                            if duration:
                                start_percent = (start_time / duration) * 100
                                end_percent = (end_time / duration) * 100
                            else:
                                start_percent = 0
                                end_percent = 100
                            
                            annotation = {
                                "id": f"emotion_{person_idx}_{emotion}_{start_frame}_{end_frame}",
                                "from_name": "events",
                                "to_name": "video",
                                "type": "labels",
                                "value": {
                                    "start": start_percent,
                                    "end": end_percent,
                                    "labels": [f"Emotion: {emotion}"],
                                    "text": [f"Person {person_idx}: {emotion}"]
                                }
                            }
                            results.append(annotation)
                            
        print(f"Prepared {len(results)} face emotion annotations for {video_id}")
        return results
    except Exception as e:
        print(f"Error preparing face annotations for {video_id}: {e}")
        return []

In [ ]:
# Function to prepare speaker annotations from Pyannote results
def prepare_speaker_annotations(video_id):
    """Convert speaker diarization data from Pyannote to Label Studio format"""
    # Check if we have any speaker diarization data from Pyannote
    # For now we'll assume it might be in the speech data since we don't have specific Pyannote column
    try:
        # This would need to be adapted to your actual data structure
        speaker_data = None
        # If you have speaker diarization data in a specific format, load it here
        # For example: speaker_data = getPyannoteData(processedvideos, video_id)  # Fictional function
        
        # If no speaker data is available, return empty results
        if not speaker_data:
            print(f"No speaker diarization data found for {video_id}")
            return []
            
        # Process the speaker data to Label Studio format
        results = []
        # Code to convert speaker segments to Label Studio format would go here
        # Similar to the other convert functions above
        
        print(f"Prepared {len(results)} speaker annotations for {video_id}")
        return results
    except Exception as e:
        print(f"Error preparing speaker annotations for {video_id}: {e}")
        return []

In [ ]:
# Process all videos in the project to add their annotations to Label Studio
def upload_all_annotations(project_id):
    """Upload all annotations for all videos in the project"""
    # Get list of videos from processedvideos DataFrame
    video_ids = processedvideos['VideoID'].unique()
    print(f"Uploading annotations for {len(video_ids)} videos")
    
    results = []
    for video_id in video_ids:
        print(f"\nProcessing {video_id}...")
        result = upload_ai_annotations_to_labelstudio(video_id, project_id)
        results.append((video_id, result))
        
    return results

In [ ]:
import json
import datetime
import traceback
from typing import List, Dict, Any, Optional

# Main function to prepare and upload all AI annotations for a video
def upload_ai_annotations_to_labelstudio(video_id, project_id):
    """Main function to prepare and upload all AI annotations for a video to Label Studio"""
    try:
        # First, get the task ID for this video in Label Studio
        tasks = ls.tasks.list(project=project_id)
        
        # Find the task that corresponds to our video
        task_id = None
        for task in tasks.items:
            if video_id in task.data.get('video_filename', ''):
                task_id = task.id
                break
        
        if not task_id:
            print(f"No task found for video {video_id}")
            return None
            
        print(f"Found task ID {task_id} for video {video_id}")
        
        # Prepare all annotations
        annotations = []
        
        # Get speech/whisper annotations
        speech_results = prepare_speech_annotations(video_id)
        if speech_results:
            annotations.extend(speech_results)
            
        # Get pose/YOLO annotations
        pose_results = prepare_pose_annotations(video_id)
        if pose_results:
            annotations.extend(pose_results)
            
        # Get face/emotion annotations
        face_results = prepare_face_annotations(video_id)
        if face_results:
            annotations.extend(face_results)
            
        # Get speaking/pyannote annotations
        speaker_results = prepare_speaker_annotations(video_id)
        if speaker_results:
            annotations.extend(speaker_results)
        
        # Combine all annotations into the proper Label Studio format
        if annotations:
            try:
                # Create annotation using the SDK's method
                response = ls.annotations.create(
                    task_id=task_id,
                    result=annotations,
                    ground_truth=False,  # These are machine annotations, not GT
                    lead_time=0,  # Auto-annotations have no lead time
                    was_cancelled=False,
                )
                print(f"Annotations uploaded successfully for {video_id}")
                return response
            except Exception as e:
                print(f"Error creating annotation: {e}")
                print(f"Annotation data structure: {json.dumps(annotations[:1], indent=2)}")



            return None
    except Exception as e:
        print(f"Error uploading annotations for {video_id}: {e}")
        print(traceback.format_exc())  # Print full traceback
        return None

## 8.5 Upload Annotations to Label Studio

Let's use our functions to upload all the AI annotations to our Label Studio project for visualization.

In [ ]:
# Add these debugging tools first
def debug_label_studio_sdk():
    """Print information about the Label Studio SDK to help diagnose issues"""
    print("Label Studio SDK Version Information:")
    try:
        import label_studio_sdk
        print(f"  Package version: {label_studio_sdk.__version__ if hasattr(label_studio_sdk, '__version__') else 'Unknown'}")
    except ImportError:
        print("  Package not properly installed")
    
    print("\nAvailable methods in LabelStudio client:")
    if 'ls' in globals():  # Check if ls is defined
        methods = [method for method in dir(ls) if not method.startswith('_')]
        print(f"  {', '.join(methods)}")
        
        print("\nAvailable attributes:")
        for attr in ['projects', 'tasks', 'annotations']:
            if hasattr(ls, attr):
                print(f"  {attr}: {type(getattr(ls, attr))}")
            else:
                print(f"  {attr}: Not available")
    else:
        print("  LabelStudio client not initialized")

# Run the debugging function
debug_label_studio_sdk()

# Try uploading annotations for a sample video with improved error handling
sample_video = processedvideos['VideoID'].iloc[0] if not processedvideos.empty else None
if sample_video:
    print(f"\nUploading annotations for sample video: {sample_video}")
    result = upload_ai_annotations_to_labelstudio(sample_video, project_id)
    if result:
        print("Upload successful. Result:", result)
    else:
        print("Upload failed. Check error messages above.")
else:
    print("No videos found in processedvideos")

In [ ]:
# Uncomment this line to upload annotations for all videos
# all_results = upload_all_annotations(project_id)

## 8.6 View Annotations in Label Studio

Now you can open the Label Studio web interface at http://localhost:8080 and navigate to your project to view the videos with their AI annotations. You'll see:

1. Speech transcriptions from Whisper
2. Person pose detections from YOLO
3. Face emotion detections from DeepFace
4. (If available) Speaker diarization from Pyannote

Each type of annotation is displayed as a temporal region in the video timeline, making it easy to see what's happening when in the video.

## 8.3 Test File Access Methods

Let's test both file access methods to make sure Label Studio can see the videos.

In [ ]:
def verify_file_access(project_id, video_path=None):
    """Test file access methods to ensure Label Studio can see videos"""
    print("Testing Label Studio file access...\n")
    
    # If no specific video provided, use the first video from processedvideos
    if video_path is None:
        if 'video_path' in processedvideos.columns and not processedvideos.empty:
            video_path = processedvideos['video_path'].iloc[0]
        else:
            print("No video paths available in processedvideos")
            return
    
    # Get the absolute path and filename
    video_path = os.path.abspath(video_path)
    video_filename = os.path.basename(video_path)
    
    print(f"Testing access to: {video_filename}")
    print(f"Full path: {video_path}\n")
    
    # METHOD 1: Test local file serving access
    print("METHOD 1: Using local file serving")
    print("-------------------------------")
    
    # Construct the URL as it would appear in Label Studio
    container_video_path = f"/label-studio/data/videos/{video_filename}"
    local_file_url = f"/data/local-files/?d={container_video_path}"
    
    print(f"URL in Label Studio: {local_file_url}")
    print("")
    print("To verify this works:")
    print("1. Open Label Studio at http://localhost:8080")
    print("2. Create a task with the above URL")
    print("3. Check if the video plays correctly")
    print("\n")
    
    # METHOD 2: Test direct file upload
    print("METHOD 2: Using direct file upload")
    print("-------------------------------")
    print("This will upload a single video file to Label Studio")
    
    # If you want to actually test an upload:
    response = input("Do you want to upload this file to test? (yes/no): ")
    if response.lower() == 'yes':
        result = upload_video_to_labelstudio(video_path, project_id, ls)
        if result:
            upload_response, task = result
            print(f"Upload successful. File URL: {upload_response.url}")
            print(f"Task created with ID: {task.id}")
    else:
        print("Upload test skipped.")
        
    print("\n")
    print("CONCLUSION:")
    print("If Method 1 works, you can use local file serving, which is faster and saves storage.")
    print("If Method 2 works, you can upload files, which is more reliable but uses more storage.")

# Run the verification with the first video
verify_file_access(project_id)

## 8.4 Import Videos Using Your Preferred Method

Now we'll import videos using either the local file serving method or the direct upload method, based on your preference.

In [ ]:
def import_videos_to_labelstudio(project_id, method='local', limit=None):
    """Import videos to Label Studio using the specified method"""
    if method not in ['local', 'upload']:
        print("Invalid method. Use 'local' for local file serving or 'upload' for direct upload.")
        return
        
    print(f"Importing videos using {method} method...")
    
    # Make sure video_path column exists in processedvideos
    if 'video_path' not in processedvideos.columns:
        print("Adding video_path column to processedvideos DataFrame")
        processedvideos['video_path'] = processedvideos['VideoID'].apply(
            lambda x: os.path.join(videos_in, x) if x.endswith('.mp4') else os.path.join(videos_in, f"{x}.mp4")
        )
        
    # If limit is specified, only use that many videos
    videos_df = processedvideos.head(limit) if limit else processedvideos
    
    # Get existing tasks to avoid duplicates
    existing_tasks = []
    try:
        tasks_response = ls.tasks.list(project=project_id)
        if tasks_response and hasattr(tasks_response, 'items'):
            existing_tasks = tasks_response.items
            print(f"Project already has {len(existing_tasks)} tasks.")
        else:
            print("No existing tasks found.")
    except Exception as e:
        print(f"Error listing tasks: {e}")
        print(traceback.format_exc())
    
    # If there are existing tasks, ask user if they want to continue
    if existing_tasks:
        response = input("Continue with import? This might create duplicates. (yes/no): ")
        if response.lower() != 'yes':
            print("Import cancelled.")
            return
    
    if method == 'local':
        # Use local file serving method
        # Prepare task data
        tasks = prepare_video_tasks(videos_df, use_local_files=True)
        print(f"Prepared {len(tasks)} video tasks for import")
        
        # Import tasks one by one
        successful = 0
        for task_data in tasks:
            try:
                response = ls.tasks.create(
                    project=project_id,
                    data=task_data['data']
                )
                successful += 1
                print(f"Imported task {successful}/{len(tasks)}: {task_data['data'].get('video_filename')}")
            except Exception as e:
                print(f"Error importing task: {e}")
                
        print(f"Successfully imported {successful} of {len(tasks)} videos using local file serving.")
        
    elif method == 'upload':
        # Use direct upload method
        print("Starting direct file uploads...")
        successful = 0
        
        for i, row in videos_df.iterrows():
            try:
                video_path = row['video_path']
                if not os.path.exists(video_path):
                    print(f"WARNING: Video file not found: {video_path}")
                    continue
                    
                result = upload_video_to_labelstudio(video_path, project_id, ls)
                if result:
                    successful += 1
                    print(f"Uploaded {successful}/{len(videos_df)}: {os.path.basename(video_path)}")
                    
                # Add a small delay between uploads
                time.sleep(1)
            except Exception as e:
                print(f"Error uploading file: {e}")
                
        print(f"Successfully uploaded {successful} of {len(videos_df)} videos.")

# Example usage (commented out by default)
# import_videos_to_labelstudio(project_id, method='local', limit=5)

In [ ]:
# Choose your preferred import method and run

# Option 1: Import using local file serving (fast, no storage used)
# Uncomment the next line to import 5 videos using local file serving:
# import_videos_to_labelstudio(project_id, method='local', limit=5)

# Option 2: Import using direct upload (more reliable, uses storage)
# Uncomment the next line to import 5 videos by uploading them:
# import_videos_to_labelstudio(project_id, method='upload', limit=5)